**1. simple-xamr parsing for chinese-english recipes**

This notebook contains code on how to run the simple-xamr parser. Running the whole notebook may take some time if you have a lot of data; remember that you need to interact with google colab to keep your session alive.

First we need to clone the repository and install the dependencies. Run these cells in order (note it takes about ~10 minutes to install required packages on google colab) and ~3 min to download the pretrained model

*input: newline separated plaintext recipes | 
output: parsed AMR trees in .txt file*

In [ ]:
import os
os.chdir('/content/')

In [ ]:
!git clone https://github.com/Yrgarcia/Cross-Lingual-AMR-Parsing

In [ ]:
import os
os.chdir('/content/Cross-Lingual-AMR-Parsing/')
!wget https://github.com/bjascob/amrlib-models/releases/download/model_parse_t5-v0_1_0/model_parse_t5-v0_1_0.tar.gz
!tar -xvzf model_parse_t5-v0_1_0.tar.gz

In [ ]:
!pip3 install -r requirements.txt
!pip install unidecode

Now that all dependencies are installed and the repo is set up, we can actually parse the AMR trees!

Note that the existing github implementation contains several hard-coded lines of logic, and I'd recommend you use my version of nmt_english.py and x_parse.py (available on github at the same location of this notebook) if you are running on google colab.

In [ ]:
# required: change to directory where files are located that you want to parse
os.chdir('/content/Cross-Lingual-AMR-Parsing/')

Note: Don't worry about output errors when the code tries to compute the SMATCH score. Since we need to generate the gold trees ourselves, we will calculate it later during evaluation.

Before running x_parse.py below, overwrite the existing versions of nmt_english.py and x_parse.py with the provided versions.

In [ ]:
# point to relative location of x_parse script and generate the trees:
!python x_parse.py -lang en -input_file sampled_golden_bloom_cn2en_adapted_ref.txt

In [ ]:
!python x_parse.py -lang en -input_file sampled_golden_bloom_cn2en_adapted_pred.txt

In [ ]:
#now let's parse the english trees
#optional change of directory if needed
os.chdir('/content/Cross-Lingual-AMR-Parsing/generation_results/en2cn/')

In [ ]:
!python x_parse.py -lang zh -input_file sampled_golden_bloom_en2cn_adapted_ref.txt

In [ ]:
!python x_parse.py -lang zh -input_file sampled_golden_bloom_en2cn_adapted_pred.txt

Note: The parsed AMR trees appear in the AMRgraphs folder.

**2. xs2match evaluation**

Now that we generated our AMR trees, it's time to evaluate them!

For this step we will use an open source cross lingual AMR evaluation tool.

*input: parsed AMR .txt file | output: xs2match score*

In [ ]:
# change to parent directory to clone the repo:
import os
os.chdir('/content/')

In [ ]:
!git clone https://github.com/shirawein/Crossling-AMR-Eval.git

In [ ]:
import os
os.chdir('/content/Crossling-AMR-Eval/s2match')

The following installations may not be necessary if you've recently ran the parsing code; however, I'm including them in case they don't exist in the runtime environment for some reason

In [ ]:
!pip install scipy
!pip install "tensorflow>=2.0.0"
!pip install --upgrade tensorflow-hub
!pip install tensorflow_text

Let's evaluate our files! First change the script permissions:

In [ ]:
!chmod +x x_evaluation-fixed-s2match.sh

Heads up: This script takes a long time to run and your colab session may get disconnected. Recommended to either install an environment where you can use the GPU properly with tensorflow or run in an environment where the session won't be getting killed.

In [ ]:
!./x_evaluation-fixed-s2match.sh /content/Cross-Lingual-AMR-Parsing/AMRgraphs/PARSABLE_sampled_golden_bloom_cn2en_adapted_pred.txt_AMR.txt /content/Cross-Lingual-AMR-Parsing/AMRgraphs/sampled_golden_bloom_cn2en_adapted_ref.txt_AMR.txt

In [ ]:
!./x_evaluation-fixed-s2match.sh /content/Cross-Lingual-AMR-Parsing/AMRgraphs/sampled_golden_bloom_en2cn_adapted_pred.txt_AMR.txt /content/Cross-Lingual-AMR-Parsing/AMRgraphs/sampled_golden_bloom_en2cn_adapted_ref.txt_AMR.txt

**3. smatch evaluation**

clone the github here: https://github.com/snowblink14/smatch and follow the readme instructions to run the smatch.py script

For precision and recall, remember to pass the --pr argument

*input: parsed AMR .txt file | output: smatch score*